In [265]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
import os
from src import utils 

# preprocessing

In [266]:
raw_path = '../data/raw/'
train = pd.read_csv(os.path.join(raw_path, 'train.csv'))
test = pd.read_csv(os.path.join(raw_path, 'test.csv'))

In [267]:
train.head()

,ID,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,...,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",ID_y,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
0,54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,...,06:00:00,0,0,0,54-102-358-02,0,0,0,0,0
1,54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,...,04:00:00,1,0,0,54-103-101-01,1,1,0,0,0
2,54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,...,07:00:00,0,0,0,54-501-026-03,0,0,0,0,0
3,54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,...,07:00:00,0,0,0,54-501-094-02,1,0,0,0,0
4,54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного о...,0,0,...,06:00:00,0,0,0,54-503-022-01,1,0,1,1,0


In [268]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955 entries, 0 to 954
Data columns (total 39 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ID                                     955 non-null    object 
 1   Пол                                    954 non-null    object 
 2   Семья                                  955 non-null    object 
 3   Этнос                                  955 non-null    object 
 4   Национальность                         955 non-null    object 
 5   Религия                                955 non-null    object 
 6   Образование                            955 non-null    object 
 7   Профессия                              955 non-null    object 
 8   Вы работаете?                          955 non-null    int64  
 9   Выход на пенсию                        955 non-null    int64  
 10  Прекращение работы по болезни          955 non-null    int64  
 11  Сахарн

In [269]:
TARGET_COLS = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']
ID_COL = 'ID'
EDU_COL = 'Образование'
SEX_COL = 'Пол'
CAT_COLS = [
    'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 
    'Профессия', 'Статус Курения', 'Частота пасс кур', 'Алкоголь',
    'Время засыпания', 'Время пробуждения'
]
OHE_COLS = [
    'Пол', 'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
    'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма', 'Туберкулез легких ', 'ВИЧ/СПИД',
    'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы','Пассивное курение', 'Сон после обеда', 
    'Спорт, клубы', 'Религия, клубы'
]
REAL_COLS = ['Возраст курения', 'Сигарет в день', 'Возраст алког']

In [270]:
train[TARGET_COLS].sum(1).max()

4

In [271]:
def drop_unnecesary_id(df: pd.DataFrame) -> pd.DataFrame:
    if 'ID_y' in df.columns:
        df = df.drop('ID_y', axis=1)
    return df

In [272]:
def add_ord_edu(df: pd.DataFrame) -> pd.DataFrame:
    df[f'{EDU_COL}_ord'] = df[EDU_COL].str.slice(0, 1).astype(np.int8).values
    return df

In [273]:
train = add_ord_edu(train)
test = add_ord_edu(test)

In [274]:
train = train.set_index(ID_COL)
test = test.set_index(ID_COL)

In [275]:
def set_idx(df: pd.DataFrame, idx_col: str) -> pd.DataFrame:
    df = df.set_index(idx_col)
    return df

In [276]:
train, target = train.drop(TARGET_COLS, axis=1), train[TARGET_COLS]

In [277]:
train = drop_unnecesary_id(train)

In [278]:
def fill_sex(df: pd.DataFrame) -> pd.DataFrame:
    most_freq = df[SEX_COL].value_counts().index[0]
    df[SEX_COL] = df[SEX_COL].fillna(most_freq)
    return df

In [279]:
def cast_types(df: pd.DataFrame) -> pd.DataFrame:
    df[CAT_COLS] = df[CAT_COLS].astype('category')

    ohe_int_cols = train[OHE_COLS].select_dtypes('number').columns
    df[ohe_int_cols] = df[ohe_int_cols].astype(np.int8)

    df[REAL_COLS] = df[REAL_COLS].astype(np.float32)
    return df
    

In [280]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = set_idx(df, ID_COL)
    df = drop_unnecesary_id(df)
    df = fill_sex(df)
    df = cast_types(df)
    return df
    
    

# MODELING

In [281]:
import os
import pandas as pd
import numpy as np

In [282]:
from sklearn.svm import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.metrics import *
from sklearn.impute import *
from sklearn.multioutput import *

In [283]:
import src.config as cfg

In [284]:
processed_data_path = '../data/processed'
train = pd.read_pickle(os.path.join(processed_data_path, 'train.pkl'))
target = pd.read_pickle(os.path.join(processed_data_path, 'target.pkl'))

Метрики

Для реализации лабораторной работы(для мультилейбла) была выбрана метрика precision - доля объектов, названная классификатором положительными, при этом являющимися положительными. Также можно найти Recall - это доля объектов, которые мы предсказали правильно, являющаяся действительно правильной. Но в решении данной задачи более важная метрика является Recall , чтобы не пропустить данные. Если брать взвешенную оценку precision и recall, то можно попробовать найти f-beta, но следует отдавать большее значение recall.

In [285]:
target.head()

,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
ID,,,,,
54-102-358-02,0,0,0,0,0
54-103-101-01,1,1,0,0,0
54-501-026-03,0,0,0,0,0
54-501-094-02,1,0,0,0,0
54-503-022-01,1,0,1,1,0


In [286]:
target.sum() / len(target)

Артериальная гипертензия              0.467016
ОНМК                                  0.042932
Стенокардия, ИБС, инфаркт миокарда    0.122513
Сердечная недостаточность             0.100524
Прочие заболевания сердца             0.090052
dtype: float64

In [287]:
y_true = np.array([1] * 9 +[0])
y_pred = np.array([0]*2 +[1]*8)

In [288]:
np.sum((y_true == y_pred)) / len(y_true)

0.7

In [289]:
#Precision
mask = y_pred == True
np.sum(y_true[mask] == y_pred[mask] )/ np.sum(mask)

0.875

In [290]:
#Recall
mask = y_true == True
np.sum(y_true[mask] == y_pred[mask] )/ np.sum(mask)

0.7777777777777778

In [291]:
recall_score(y_true,y_pred)

0.7777777777777778

In [292]:
from functools import partial

In [293]:
scoping = partial(fbeta_score, beta=2.0)

In [294]:
scoping(y_true,y_pred)

0.7954545454545454

Валидационная выборка

In [295]:
import pickle

In [296]:
RS = 77

In [297]:
train = pd.read_pickle(os.path.join(processed_data_path, 'train.pkl'))
target = pd.read_pickle(os.path.join(processed_data_path, 'target.pkl'))

In [298]:
train[cfg.CAT_COLS] = train[cfg.CAT_COLS].astype('object')

In [299]:
train_data, val_data, train_target, val_target = train_test_split(train, target, train_size=0.8, random_state=RS)

In [300]:
train_idx, val_idx = train_test_split(
        train.index, test_size=0.2, random_state=7)

In [301]:
val_idx.to_frame().to_pickle('temp.pkl')

In [302]:
with open('tmp2.pkl', 'wb') as f:
    pickle.dump(val_idx, f)

In [303]:
with open('tmp2.pkl', 'rb') as f:
    val_idx2 = pickle.load(f)

In [304]:
train_data.head()

,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,...,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы"
ID,,,,,,,,,,,,,,,,,,,,,
54-103-054-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,1,0,0,...,NaN,0,NaN,употребляю в настоящее время,26.0,22:00:00,06:00:00,0,1,0
54-102-299-01,Ж,вдовец / вдова,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,1,0,...,NaN,0,NaN,никогда не употреблял,NaN,22:30:00,08:30:00,0,1,0
54-102-138-01,Ж,никогда не был(а) в браке,европейская,Русские,Христианство,5 - ВУЗ,низкоквалифицированные работники,1,0,0,...,NaN,0,NaN,никогда не употреблял,NaN,22:00:00,06:30:00,0,0,0
54-601-033-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,4 - профессиональное училище,операторы и монтажники установок и машинного о...,0,1,0,...,NaN,0,NaN,употребляю в настоящее время,20.0,22:00:00,07:00:00,0,0,0
54-602-054-01,Ж,в разводе,европейская,Русские,Христианство,4 - профессиональное училище,ведение домашнего хозяйства,0,1,0,...,10.0,0,NaN,употребляю в настоящее время,18.0,00:00:00,07:00:00,1,0,0


In [305]:
real_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())
]
)

In [306]:
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [307]:
import category_encoders as ce

In [308]:
preprocess_pipe = ColumnTransformer(transformers=[
    ('real_cols', real_pipe, cfg.REAL_COLS),
    ('cat_cols', cat_pipe, cfg.CAT_COLS),
    ('woe_cat_cols',ce.WOEEncoder(),cfg.CAT_COLS),
    ('ohe_cols', 'passthrough', cfg.OHE_COLS)
]
)

In [309]:
model = LinearSVC()

In [310]:
model_pipe = Pipeline([
    ('preprocess', preprocess_pipe),
    ('model', model)
]
)

In [311]:
multiout_model_pipe = MultiOutputClassifier(model_pipe, n_jobs=4)

In [312]:
scores = cross_val_score(
    estimator=multiout_model_pipe,
    X=train_data,
    y=train_target,
    scoring='f1_samples',
    cv=3,
    n_jobs=1
)

c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [313]:
scores

array([0.24287582, 0.21764706, 0.23425197])

In [314]:
d = pd.DataFrame()
def metric(val_target, val_data, name, d = d): 

    a = accuracy_score(pd.DataFrame(val_target), val_data)
    r = recall_score(pd.DataFrame(val_target), val_data, average='samples')
    p = precision_score(pd.DataFrame(val_target), val_data, average='samples')
    f1 =f1_score(pd.DataFrame(val_target), val_data, average='samples')
    df = pd.DataFrame({"Accuracy":([a]), "Recall":([r]) , "Precision":([p]), "F1":([f1])}, index=[name])
    return df

In [315]:
train_data_transform = preprocess_pipe.fit_transform(train_data)
val_data = preprocess_pipe.transform(val_data)

In [316]:
from catboost import CatBoostClassifier, Pool

In [317]:
pool = Pool(train_data_transform, train_target)

In [318]:
catboost = CatBoostClassifier(iterations=100, loss_function='MultiLogloss', custom_metric=['Recall', "F1", "Precision"], silent=True).fit(pd.DataFrame(train_data_transform),pd.DataFrame(train_target))

In [319]:
catboost.fit(pool, verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [320]:
logistic = MultiOutputClassifier(LogisticRegression(solver="liblinear", class_weight='balanced', max_iter=100)).fit(pd.DataFrame(train_data_transform),pd.DataFrame(train_target))

In [321]:
utils.save_as_pickle(metric(val_target, catboost.predict(val_data), "CatBoost"), "../data/processed/metric_catboost.pkl")
utils.save_as_pickle(metric(val_target, logistic.predict(val_data), "LogisticRegression"), "../data/processed/metric_logistic.pkl")

3
3


c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318

In [322]:
pd.read_pickle('../data/processed/metric_catboost.pkl')

,Accuracy,Recall,Precision,F1
CatBoost,0.513089,0.267888,0.356021,0.293019


In [323]:
pd.read_pickle('../data/processed/metric_logistic.pkl')

,Accuracy,Recall,Precision,F1
LogisticRegression,0.235602,0.367801,0.207243,0.248604


In [324]:
import os
import time
from src import utils

model_path = '../models'
report_path = '../reports'
pp_catboost = utils.save_as_pickle(catboost, os.path.join(model_path, 'catboost.pkl'))
print(os.path.join(model_path, 'catboost.pkl'))

1
../models\catboost.pkl


INFERENCE

In [325]:
model_path = '../models'
report_path = '../reports'
pp = utils.load_model(os.path.join(model_path, 'catboost.pkl'))

In [326]:
processed_data_path = '../data/processed/'
test = pd.read_pickle(os.path.join(processed_data_path, 'test.pkl'))

In [327]:
models_cb = [
    'catboost',
]

In [328]:
for name in models_cb:
    model = utils.load_model(os.path.join(model_path, name + '.pkl'))
    y_pred = model.predict(preprocess_pipe.transform(train))
    pd.DataFrame(y_pred).to_csv(os.path.join(report_path, f'{name}_{time.time()}.csv'))

In [329]:
cb_test = test.copy()
cb_test[cfg.REAL_COLS] = cb_test[cfg.REAL_COLS].astype(str)

In [330]:
for name in models_cb:
    model = utils.load_model(os.path.join(model_path, name + '.pkl'))
    y_pred = model.predict(val_data)
    pd.DataFrame(y_pred).to_csv(os.path.join(report_path, f'{name}_{time.time()}.csv'))